In [1]:
!pip install langchain-chroma langchain pandas pypdf langchain-openai

In [25]:
!pip install --upgrade langchain langchain-community

In [47]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature = 0,
)

In [48]:
from langchain.document_loaders import PyPDFLoader
docs = [] # 추출된 문서를 저장할 리스트를 초기화

loader = PyPDFLoader("d:/data/안구건조증.pdf") # "d:/data/안구건조증.pdf" 경로의 PDF 파일을 PyPDFLoader로 로드

# PDF에서 텍스트 데이터를 추출하여 리스트 형태로 반환
text_content = loader.load()

In [49]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma # 벡터 데이터를 저장하고 검색하는 벡터 데이터베이스 (ChromaDB)
from langchain.text_splitter import CharacterTextSplitter

# 문서 분할 (텍스트를 작은 단위로 나누기)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

# 문서를 CharacterTextSplitter를 사용하여 분할
splits = text_splitter.split_documents(text_content)
# 문서를 CharacterTextSplitter를 사용하여 분할
texts = [doc.page_content for doc in splits]

# 문서를 CharacterTextSplitter를 사용하여 분할
embeddings = OpenAIEmbeddings()

# 변환된 벡터를 Chroma 벡터 데이터베이스에 저장
vectorstore = Chroma(persist_directory='d:/data/db_chroma_complete', embedding_function=embeddings)

In [51]:
from langchain_core.prompts import PromptTemplate
custom_prompt_template = PromptTemplate(
    template = """당신은 안구건조증에 대한 정보를 제공하는 건강 지킴이 챗봇입니다.

{context}

제공된 소스 데이터만 사용하여 고객의 질문에 답하세요.
모르겠다면 "잘 모르겠습니다."라고 답하세요.
친절하고 예의 바르며 전문적인 어조를 사용하세요.
답변은 간결하게 유지하세요.

질문: {question}

답변:
""",
input_variables=["context"]
)
chain_type_kwargs = {"prompt": custom_prompt_template}

In [52]:
query = "습도가 안구건조증에 미치는 영향은?"
retriever = vectorstore.similarity_search(query,k=8) # vectorstore(ChromaDB 벡터 저장소)에서 검색 수행

In [54]:
from langchain.chains import RetrievalQA

# RetrievalQA.from_chain_type()을 사용하여 QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 기본적으로 retrieved 문서를 합쳐서 처리
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs=chain_type_kwargs   # 
)

# 사용자의 질문을 입력하여 검색 및 응답 생성
result = qa_chain.invoke({"query": query})
result["result"]

'습도는 안구건조증에 중요한 영향을 미칩니다. 낮은 습도 환경에서는 눈의 수분 증발이 증가하여 안구건조증 증상이 악화될 수 있습니다. 적절한 실내 습도를 유지하는 것이 증상 완화에 도움이 될 수 있습니다.'

In [55]:
query = "인공눈물은 자주 사용해도 되는거야?"
result = qa_chain.invoke({"query": query})
result["result"]

'인공눈물은 일반적으로 안전하게 자주 사용할 수 있습니다. 그러나 하루에 여러 번 사용해야 할 경우, 방부제가 없는 제품을 선택하는 것이 좋습니다. 만약 증상이 지속되거나 악화된다면, 안과 전문의와 상담하는 것이 좋습니다.'

In [56]:
# 메모리 관리
from langchain.memory import ConversationBufferMemory # 랭체인에서 대화 이력을 저장하고 관리하는 라이브러리

memory = ConversationBufferMemory(
    memory_key="chat_history", # 저장된 대화를 "chat_history"라는 키 값으로 관리
    return_messages=True, # 이전 대화를 메시지 형식(List[Message])으로 반환 (AI가 대화 문맥을 이해할 수 있도록 함)
    output_key="output" # 출력 데이터를 저장할 키 값
)

In [57]:
from langchain.chains import ConversationalRetrievalChain # 문맥을 유지하면서 검색 기능을 추가한 질의응답(QA) 체인 라이브러리
retriever=vectorstore.as_retriever()
qa_chain_memory = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever, # ChromaDB의 벡터 데이터베이스에서 유사한 문서를 검색할 수 있도록 설정
    memory=memory # 이전 대화를 기억하여 문맥을 유지할 수 있도록 설정
)

In [60]:
query = "안구건조증 예방 방법은?"
result = qa_chain.invoke({"query": query})
result["result"]

'안구건조증을 예방하기 위해서는 눈을 자주 깜빡이고, 인공눈물을 사용하여 눈을 촉촉하게 유지하며, 컴퓨터나 스마트폰 사용 시 적절한 휴식을 취하는 것이 중요합니다. 또한, 실내 습도를 적절히 유지하고, 바람이 강한 환경에서는 보호 안경을 착용하는 것도 도움이 됩니다.'

In [62]:
query = "이것은 왜 생기는거야?"
result = qa_chain.invoke({"query": query})
result["result"]

'안구건조증은 눈물이 충분히 생성되지 않거나 눈물이 너무 빨리 증발할 때 발생할 수 있습니다. 이는 눈의 표면을 보호하고 윤활하는 데 필요한 눈물의 부족으로 인해 발생합니다. 다양한 요인들이 원인이 될 수 있으며, 환경적 요인, 장시간의 컴퓨터 사용, 노화, 특정 약물 등이 포함될 수 있습니다.'